In [1]:
import requests 
from bs4 import BeautifulSoup
import json
import csv

In [3]:
class ZillowScraper:
    results = []
    headers = {
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,ru;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': 'zguid=23|%240bc9d364-2ef6-4c88-a0d3-863ec03ebaf7; zgsession=1|0193e537-a954-42fe-983e-3bb4cc763f49; G_ENABLED_IDPS=google; JSESSIONID=18C184C96563EB819DB20AAC2E6CDE2A; AWSALB=jjU22Ho6o5A4kcNyakgiONZjT3uwJs7uyki4k5DgXgqFD9Y5DwIrIp/U7I6tK5H9v7hWJ6rZMjBqdPJ4coIi5M5lNv1zyRVmEqaGBWIg3TLjJ/wuxTblMRC6rNus; AWSALBCORS=jjU22Ho6o5A4kcNyakgiONZjT3uwJs7uyki4k5DgXgqFD9Y5DwIrIp/U7I6tK5H9v7hWJ6rZMjBqdPJ4coIi5M5lNv1zyRVmEqaGBWIg3TLjJ/wuxTblMRC6rNus; search=6|1589478620708%7Crect%3D40.82416022149176%252C-73.53421215649415%252C40.51819422545078%252C-74.12850384350587%26rid%3D270915%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D0%26fs%3D0%26fr%3D1%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%09%09270915%09%09%09%09%09%09',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (iPad; CPU OS 11_0 like Mac OS X) AppleWebKit/604.1.34 (KHTML, like Gecko) Version/11.0 Mobile/15A5341f Safari/604.1'
        }
   
    def fetch(self, url,params):
        print('HTTP GET request to URL: %s'%url, end='')
        res = requests.get(url , params = params, headers = self.headers)
        print(' | Status code: %s' % res.status_code)
        
        return res
        
    def save_response(self, res):
        with open('res.html', 'w',encoding='utf-8') as html_file:
            html_file.write(res)
    
    def load_response(self):
        html = ''
        
        with open('res.html','r',encoding='utf-8') as html_file:
            for line in html_file:
                html+=line
        return html
    
    def parse(self,html):
        content = BeautifulSoup(html, 'lxml')
        cards = content.findAll('article',{'class':'list-card'})
            
        for card in cards:
            try:
                squareFeet = card.find('ul',{'class':'list-card-details'}).findAll('li')[2].text.split(' ')[0]
            except: 
                squareFeet = 'N/A'
                
            if(card.find('ul',{'class':'list-card-details'}).find('li').text.startswith( '$' )):
                price = card.find('ul',{'class':'list-card-details'}).find('li').text.split(' ')[0].replace("+", "")
                bathrooms = "1"
                squareFeet = 'N/A'
                try:
                    bedroom = card.find('ul',{'class':'list-card-details'}).find('li').text.split('+ ')[1].split(' ')[0]
                except:
                    bedroom = card.find('ul',{'class':'list-card-details'}).find('li').text.split(' ')[1].split(' ')[0]
            else:
                price = card.find('div',{'class':'list-card-price'}).text.split('/')[0]
                bedroom = card.find('ul',{'class':'list-card-details'}).find('li').text.split(' ')[0]
                bathrooms = card.find('ul',{'class':'list-card-details'}).findAll('li')[1].text.split(' ')[0]
                try:
                    squareFeet = card.find('ul',{'class':'list-card-details'}).findAll('li')[2].text.split(' ')[0]
                except: 
                    squareFeet = 'N/A'
                
            if ('| ') in card.find('address',{'class':'list-card-addr'}).text:
                address = card.find('address',{'class':'list-card-addr'}).text.split('| ')[1]
            else:
                address = card.find('address',{'class':'list-card-addr'}).text
                
            self.results.append({
                'price': price,
                'address':address,
                'bedrooms':bedroom,
                'bathrooms':bathrooms,
                 'squareFeet':squareFeet
            })
    def to_csv(self):
        with open('zillowQueens.csv','w') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.results[0].keys(),lineterminator = '\n')
            writer.writeheader()
            for row in self.results:
                writer.writerow(row)

    def run(self):
        firstPage = True
        pPage = 2
        pageNum = 3
        for page in range(1,15):
            params = {
                'searchQueryState': '{"pagination":{"currentPage":%s},"usersSearchTerm":"Queens, New York, NY","mapBounds":{"west":-74.12850384350587,"east":-73.53421215649415,"south":40.51819422545078,"north":40.82416022149176},"regionSelection":[{"regionId":270915,"regionType":17}],"isListVisible":true,"isMapVisible":false,"mapZoom":12,"filterState":{"pmf":{"value":false},"fore":{"value":false},"auc":{"value":false},"nc":{"value":false},"fr":{"value":true},"fsbo":{"value":false},"cmsn":{"value":false},"pf":{"value":false},"fsba":{"value":false}}}'%page             
            }
            res = self.fetch('https://www.zillow.com/queens-new-york-ny/rentals/',params)
            self.parse(res.text)
            self.to_csv()
#                 firstPage = False
#             else:
#                 params = {
#                     'searchQueryState': '{"pagination":{"currentPage":'+str(pageNum)+',"usersSearchTerm":"Brooklyn, New York, NY","mapBounds":{"west":-74.26086287280273,"east":-73.61438612719726,"south":40.50200122440905,"north":40.80804134081856},"mapZoom":12,"regionSelection":[{"regionId":37607,"regionType":17}],"filterState":{"pmf":{"value":false},"fore":{"value":false},"auc":{"value":false},"nc":{"value":false},"fr":{"value":true},"fsbo":{"value":false},"cmsn":{"value":false},"pf":{"value":false},"fsba":{"value":false}},"isListVisible":true,"isMapVisible":false}'
#                          }
#                 res = self.fetch('https://www.zillow.com/brooklyn-new-york-ny/rentals/'+str(pageNum)+'_p/',params)
#                 self.parse(res.text)
#                 self.to_csv()
#                 pageNum = int(pageNum) + 1
#                 pPage = int(pPage) + 1
        
if __name__ == '__main__':
    
    scraper = ZillowScraper()
    scraper.run()

HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/queens-new-york-ny/rentals/ | Status code: 200
HTTP GET request to URL: https://www.zillow.com/qu